In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
g1 = tf.Graph()
with g1.as_default() as graph:
    
    nb_classes = 10
    
    rate = tf.compat.v1.placeholder(tf.float32)
    
    X = tf.compat.v1.placeholder(tf.float32, [None, 784])
    X_img = tf. reshape(X, [-1, 28, 28, 1])
    Y = tf.compat.v1.placeholder(tf.float32, [None, nb_classes])
    
    W1 = tf.Variable(tf.compat.v1.random_normal([3, 3, 1, 32], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L1 = tf.nn.dropout(L1, rate=rate)
    
    W2 = tf.Variable(tf.compat.v1.random_normal([3, 3, 32, 64], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L2 = tf.nn.dropout(L2, rate=rate)
    
    W3 = tf.Variable(tf.compat.v1.random_normal([3, 3, 64, 128], stddev=0.01))
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L3 = tf.nn.dropout(L3, rate=rate)
    L3 = tf.reshape(L3, [-1, 128 * 4 * 4])    
    
    W4 = tf.compat.v1.get_variable("W4", shape = [128 * 4 * 4, 625], initializer=tf.initializers.glorot_normal())
    b4 = tf.Variable(tf.compat.v1.random_normal([625]))
    L4 = tf.matmul(L3, W4) + b4
    L4 = tf.nn.dropout(L4, rate=rate)
    
    W5 = tf.compat.v1.get_variable("W5", shape = [625, nb_classes], initializer=tf.initializers.glorot_normal())
    b5 = tf.Variable(tf.compat.v1.random_normal([nb_classes]))
    hypothesis = tf.matmul(L4, W5) + b5
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))    
    
    init_op = tf.compat.v1.global_variables_initializer()
    
    r = random.randint(0, mnist.test.num_examples - 1)
    test = tf.argmax(mnist.test.labels[r:r+1], 1)
    
sess = tf.compat.v1.Session(graph = g1)
sess.run(init_op)

In [5]:
training_epocs = 15
batch_size = 100

for epoch in range(training_epocs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, rate: 0.3})
        avg_cost += c / total_batch
    print('Epoch : ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))

Epoch :  0001 cost =  0.488297633
Epoch :  0002 cost =  0.085672562
Epoch :  0003 cost =  0.065891067
Epoch :  0004 cost =  0.055263868
Epoch :  0005 cost =  0.051024040
Epoch :  0006 cost =  0.047943121
Epoch :  0007 cost =  0.044401057
Epoch :  0008 cost =  0.040706333
Epoch :  0009 cost =  0.040042286
Epoch :  0010 cost =  0.036804013
Epoch :  0011 cost =  0.036457565
Epoch :  0012 cost =  0.036975179
Epoch :  0013 cost =  0.035737303
Epoch :  0014 cost =  0.032188744
Epoch :  0015 cost =  0.032164699


In [6]:
## 1에폭당 4분 50초
acc_sum = 0

for i in range(2):
    acc_sum += sess.run(accuracy, feed_dict={X: mnist.test.images[5000 * i:5000 * (i + 1)], Y: mnist.test.labels[5000 * i:5000 * (i + 1)], rate: 0.0})
    
acc = acc_sum / 2.0    
print("Accuracy:", acc)    

Accuracy: 0.9931999742984772
